In [1]:
import glob
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.common.features.feature_metadata import FeatureMetadata

In [2]:
training_files = 3
data_files = glob.glob('play_data/*.tsv')
dfs = [pd.read_csv(file, sep='\t') for file in data_files[:training_files]]

train_data = TabularDataset(pd.concat(dfs, ignore_index=True))
train_data['play_pos'] = train_data['trick_num'] * 4 + train_data['play_seq']

In [3]:
train_data.head()

,run_id,key,trick_num,play_seq,pos,partner_winning,lead_trumped,tricks_won,trumps_seen,turn_seen,...,throw_off_long,throw_off_low,throw_off_next,tricks_min,tricks_max,tricks_avg,points_min,points_max,points_avg,play_pos
0,953904,10♣ 9♥ A♣ J♥ K♣,5,1,0,0,0,1,6,3,...,0,0,0,1,1,1.0,-2,-2,-2.0,21
1,953904,10♣ 9♥ J♥ K♣ A♣,5,1,0,0,0,0,6,3,...,0,0,0,1,1,1.0,-2,-2,-2.0,21
2,953904,9♥ 10♣ J♥ A♣ K♣,5,0,0,0,0,1,5,3,...,0,0,0,2,2,2.0,-2,-2,-2.0,20
3,953904,10♣ J♥ 9♥ A♣ K♣,5,1,0,0,0,0,6,3,...,0,0,0,1,1,1.0,-2,-2,-2.0,21
4,953904,10♣ J♥ K♣ 9♥ A♣,5,1,0,0,0,1,6,3,...,0,0,0,1,1,1.0,-2,-2,-2.0,21


In [4]:
label = 'points_avg'
train_data[label].describe()

count    587716.000000
mean         -0.450098
std           1.969065
min          -4.000000
25%          -2.000000
50%          -1.000000
75%           1.000000
max           4.000000
Name: points_avg, dtype: float64

In [5]:
exclude = ['run_id', 'key', 'trick_num', 'play_seq']
outputs = ['tricks_min', 'tricks_max', 'tricks_avg', 'points_min', 'points_max', 'points_avg']
outputs.remove(label)

type_map = {k: 'int' for k in list(train_data)}
type_map['play_pos'] = 'category'
type_map[label] = 'float'
metadata = FeatureMetadata(type_map_raw=type_map)

predictor = TabularPredictor(problem_type='regression', label=label)
predictor.fit(train_data.drop(columns=(exclude + outputs)), feature_metadata=metadata, presets='best_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20240904_191812"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.9
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #202405300957~1722440358~24.04~aa0a43f~dev-Ubuntu SMP PREEMPT_DY
CPU Count:          8
Memory Avail:       8.63 GB / 15.34 GB (56.3%)
Disk Space Avail:   119.96 GB / 452.95 GB (26.5%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoG

(_ray_fit pid=34273) [1000]	valid_set's rmse: 0.844531
(_ray_fit pid=34269) [1000]	valid_set's rmse: 0.841131 [repeated 6x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)


(_dystack pid=33066) 	-0.84	 = Validation score   (-root_mean_squared_error)
(_dystack pid=33066) 	124.16s	 = Training   runtime
(_dystack pid=33066) 	251.59s	 = Validation runtime
(_dystack pid=33066) Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.0s of the 287.22s of remaining time.
(_dystack pid=33066) 	Ensemble Weights: {'KNeighborsUnif_BAG_L1': 0.706, 'LightGBMXT_BAG_L1': 0.294}
(_dystack pid=33066) 	-0.6555	 = Validation score   (-root_mean_squared_error)
(_dystack pid=33066) 	0.51s	 = Training   runtime
(_dystack pid=33066) 	0.01s	 = Validation runtime
(_dystack pid=33066) Fitting 106 L2 models ...
(_dystack pid=33066) Fitting model: LightGBMXT_BAG_L2 ... Training model for up to 286.68s of the 286.64s of remaining time.
(_dystack pid=33066) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=9.99%)


(_ray_fit pid=34916) [1000]	valid_set's rmse: 0.564141
(_ray_fit pid=34923) [1000]	valid_set's rmse: 0.57251
(_ray_fit pid=34916) [2000]	valid_set's rmse: 0.551386 [repeated 7x across cluster]


(_dystack pid=33066) 	-0.5568	 = Validation score   (-root_mean_squared_error)
(_dystack pid=33066) 	231.38s	 = Training   runtime
(_dystack pid=33066) 	652.59s	 = Validation runtime
(_dystack pid=33066) Fitting model: WeightedEnsemble_L3 ... Training model for up to 360.0s of the -31.71s of remaining time.
(_dystack pid=33066) 	Ensemble Weights: {'LightGBMXT_BAG_L2': 1.0}
(_dystack pid=33066) 	-0.5568	 = Validation score   (-root_mean_squared_error)
(_dystack pid=33066) 	0.36s	 = Training   runtime
(_dystack pid=33066) 	0.01s	 = Validation runtime
(_dystack pid=33066) AutoGluon training complete, total runtime = 927.72s ... Best model: WeightedEnsemble_L3 | Estimated inference throughput: 68.1 rows/s (65302 batch size)
(_dystack pid=33066) TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20240904_191812/ds_sub_fit/sub_fit_ho")
(_dystack pid=33066) Deleting DyStack predictor artifacts (clean_up_fits=True) ...
Leaderboard on holdout data (DySta

In [6]:
test_data = TabularDataset(data_files[training_files])
test_data['play_pos'] = test_data['trick_num'] * 4 + test_data['play_seq']

y_pred = predictor.predict(test_data.drop(columns=([label] + exclude + outputs)))
y_pred.head()

Loaded data from: play_data/play_model_1-20240903_111103.tsv | Columns = 56 / 56 | Rows = 192843 -> 192843


0    1.002749
1   -1.978051
2   -1.133849
3    1.031326
4    1.003660
Name: points_avg, dtype: float32

In [7]:
predictor.evaluate(test_data, silent=True)

{'root_mean_squared_error': -1.2049994406491524,
 'mean_squared_error': -1.4520236519647705,
 'mean_absolute_error': -0.7983484078161389,
 'r2': 0.5924398511405975,
 'pearsonr': 0.7748825699879294,
 'median_absolute_error': -0.4707298278808594}

In [8]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT_BAG_L1,-0.997960,-0.709559,root_mean_squared_error,263.304084,1508.953194,396.510878,263.304084,1508.953194,396.510878,1,True,3
1,LightGBMXT_BAG_L2,-1.204999,-0.453243,root_mean_squared_error,1023.594556,3526.883198,1113.283828,351.840312,796.811951,715.971702,2,True,5
2,WeightedEnsemble_L3,-1.204999,-0.453243,root_mean_squared_error,1023.596808,3526.888609,1113.728834,0.002252,0.005410,0.445007,3,True,6
3,WeightedEnsemble_L2,-1.288255,-0.582062,root_mean_squared_error,464.560854,2115.845948,397.465301,0.002741,0.005588,0.553734,2,True,4
4,KNeighborsUnif_BAG_L1,-1.649370,-0.698896,root_mean_squared_error,207.196131,614.230888,0.400559,207.196131,614.230888,0.400559,1,True,1
5,KNeighborsDist_BAG_L1,-1.651600,-0.633147,root_mean_squared_error,201.254030,606.887166,0.400689,201.254030,606.887166,0.400689,1,True,2
